# Visualising GeoSpatial Data with Python


Data visualization is the presentation of data in a pictorial or graphical format. This enables decision makers to see analytics presented visually, and grasp concepts which may otherwise be complex  to indentify.

In this session will guide you set-by-step on how to transform, merge and visualise spatial data or AirBnbs in Berlin.


## Charateristics of GeoSpatial Data 
- Information that identifies the geographic location of features and boundaries on Earth:
    - E.g oceans, countiries, rivers, buildings and roads
    
- **Examples of Spatial data:**    
    - **Point:** one coordinate
    - **Line:** two coordinates representing a line segment of the element. 
    - **Polygon:**  consists of coordinate pair values, one vertex pair for each 

## Source of Data

**Airbnb Listing in Berlin**
- Tabulated data with cordinates of listings
- csv formart
- Inside AirBnb

Refence: http://data.insideairbnb.com/germany/be/berlin/2019-05-14/visualisations/listings.csv 

**Map of neighborhoods in Berlin**
- Polygons representing the neighborhoods in Berlin
- GeoJson Format
- GitHub

Reference: https://github.com/funkeinteraktiv/Berlin-Geodaten/raw/master/berlin_bezirke.geojson



In [ ]:
# web:
# listings_url = 'http://data.insideairbnb.com/germany/be/berlin/2019-05-14/visualisations/listings.csv'
# berlin_geojson_url ='https://github.com/funkeinteraktiv/Berlin-Geodaten/raw/master/berlin_bezirke.geojson'

In [ ]:
# local
listings_url='data/listings.csv'
berlin_geojson_url='data/berlin_bezirke.geojson'

## Tools Used
- **python 3.4.5**
    - Encountered a number of error with Python 2.7, and for that Python 3.4.5 or higher is recommeded.

- **pandas 0.24.2**
    - import csv data into DataFrame which makes transformation and filtering easir
    
- **geopandas 0.4.1**
    - import geoSpatial data into GeoDataFrame which suports data transformation and merging multiple datasets
    
- **spaciy 1.6.4** 
    - Used in transforming data. E.g Creating spatial attribute (of type 'Point') for listing from longtidues and latitudes provided in csv. 
    
- **follium 0.9.1**
    -  To create several types of Leaflet maps Python and Leaflet.js

- **matplotlib 3.1.0**
    - embedding plots into python applications

In [ ]:
#import Prerequists
%matplotlib inline

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import folium

In [ ]:
# verify version
print(folium.__version__)

## Importing csv data with pandas

In [ ]:
df_listings = pd.read_csv(listings_url, parse_dates=True, index_col=0)
df_listings.head(2)

In [ ]:
df_listings.columns

# Filtering data 
**Criteria:**
- Entire home/apartments listings
- price above 500

In [ ]:


entire_homes_above500=df_listings[
    (df_listings.room_type == 'Entire home/apt')&
    (df_listings.price >= 500)]

entire_homes_above500.head(2)



## Add a geometry attribute

In [ ]:
entire_homes_above500.loc[:,'geometry'] = entire_homes_above500.apply(
                                                    lambda row: Point((row['longitude'], row['latitude'])), axis = 1)

#Verify coordinates
entire_homes_above500[['longitude','latitude','geometry']].head(2)

In [ ]:
print(entire_homes_above500.shape,
      type(entire_homes_above500))

In [ ]:
entire_homes_above500.crs

## Create a GeoDataFrame for listing in Mitte
The data can not be maped yet, we need to transform into a geopandas data frame with a Coordinate Reference System(CRS) that is compatiple with OpenStreetMap(epsg:4326)

In [ ]:
listings_crs = {'init': 'epsg:4326'} 

entire_homes_above500_geo = gpd.GeoDataFrame(entire_homes_above500, 
                                             crs = listings_crs, 
                                             geometry = entire_homes_above500.geometry)
entire_homes_above500_geo[['longitude','latitude','geometry']].head(2)

In [ ]:
#
print(entire_homes_above500_geo.shape,
      type(entire_homes_above500_geo),
      entire_homes_above500_geo.crs)

## Geographical Visualisations

### Folium Map Parameters
- **location** (tuple or list, default None) – Latitude and Longitude of Map (Northing, Easting).
- **width** (pixel int or percentage string (default: '100%')) – Width of the map.
- **height** (pixel int or percentage string (default: '100%')) – Height of the map.
- **tiles** (str, default 'OpenStreetMap') Other ption inclide:
    - openstreetmap : _openstreetmap_
    - Map Quest Open: _mapquestopen_
    - MapQuest Open _Aerial: MapQuest Open Aerial_
    - Mapbox Bright: _Mapbox Bright_
    - Mapbox Control _Room:Mapbox Control Room_
    - Stamen Terrain: _stamenterrain_
    - Stamen Toner: _stamentoner_
    - Stamen Watercolor: _stamenwatercolor_
    - CartoDB Positron: _cartodbdark_matter_
    
- **min_zoom** (int, default 0) – Minimum allowed zoom level for the tile layer that is created.
- **max_zoom** (int, default 18) – Maximum allowed zoom level for the tile layer that is created.
- **zoom_start** (int, default 10) – Initial zoom level for the map.
- **crs** (str, default 'EPSG3857') – Defines coordinate reference systems for projecting geographical points into pixel (screen) coordinates and back. 


In [ ]:
# To construct a map with centered Alexanderplatz Station
# Remember plotting is done by 'latitude', 'longitude (Northing, Easting)

Berlin_Alexanderplatz_Station = [52.521389, 13.411944]

m = folium.Map(location = Berlin_Alexanderplatz_Station, zoom_start = 16)

# display the map
display(m)


## Adding a marker to the map

In [ ]:
starbucks_point = {'coordinates':[52.521210, 13.411571], 'name': 'Starbucks'}
mcDonalds_point = {'coordinates':[52.52115,13.41196], 'name': 'McDonalds'}

station_map = folium.Map(location = mcDonalds_point['coordinates'],
                         zoom_start=18)

folium.Marker(location=starbucks_point['coordinates'], 
              popup=starbucks_point['name']).add_to(station_map)

display(station_map)

# Working with Polygons

## Importing data with geopandas

In [ ]:
berlin = gpd.read_file(berlin_geojson_url) 
berlin.head(2)

## Data Transformation

### Get the center of the polygon

- The central point of a shape and is also called the geometric center this is done using the **Centroid** function. 
- The central point matches to the center of gravity of a particular shape. 
- The centroid is the term for 2-dimensional shapes.
- Represented by represented with longitudes by latitudes

In [ ]:
##To add the center colum on the Berlin GeoDataFrame
berlin['center'] = berlin.geometry.centroid
berlin.head(2)


## Filtering 

In [ ]:
berlin_mitte = berlin.loc[berlin.name == 'Mitte']
berlin_mitte.head()

# Ploting a polygon
In this example we will plot only the polygon for Mitte

In [ ]:
# Get the central point for Mitte to focus the mapping area 
center_point = berlin_mitte.center[0] 
print(type(center_point), center_point)

In [ ]:
# Remember folium cordinates are latitudes by longitudes
mitte_center = [center_point.y, center_point.x]
print(mitte_center)

In [ ]:
# Construct a folium map for Mitte in Berlin
mitte_map = folium.Map(location = mitte_center, width='60', height='60', zoom_start=12)
folium.GeoJson(berlin_mitte.geometry).add_to(mitte_map)
display(mitte_map)

#  Multiple layers and customision
Scenario: Show the location of Starbucks in Berlin Alexanderplatz Station in 
- each map feature is added as separent layer


In [ ]:
mitte_map_with_one_point = folium.Map(location = mitte_center,
                                      width='80', 
                                      height='80', 
                                      tiles='Stamen Toner', 
                                      zoom_start=12)

cafes_in_Mitte = [(52.521210, 13.411571),
                      (52.51673,13.37978),
                      (52.51940,13.38874),
                      (52.5197446, 13.3886757),
                      (2.5179169, 13.3881200)]
#add ploygon to the map
folium.GeoJson(berlin_mitte.geometry).add_to(mitte_map_with_one_point)
# folium.GeoJson(roads_mitte.geometry).add_to(mitte_map_with_one_point)

#add different points to the map
folium.Circle(location=cafes_in_Mitte[0],
              radius=100, 
              color='red',
              fill_color = 'red',
              fill_opacity=0.9).add_to(mitte_map_with_one_point)

folium.CircleMarker(location=cafes_in_Mitte[1], 
                    radius=10,
                    color = None,
                    fill_color = '#FF8C00',
                    fill_opacity=0.9).add_to(mitte_map_with_one_point)

folium.Marker(location=cafes_in_Mitte[2], 
              icon = folium.Icon(color='green', icon='credit-card')).add_to(mitte_map_with_one_point)

#show the map
display(mitte_map_with_one_point)

# Interating through a DataFrame  

In [ ]:

mitte_map = folium.Map(location = mitte_center,
                       width='80', 
                       height='80', 
                       tiles='cartodbpositron',
                       zoom_start = 10)
folium.GeoJson(berlin.geometry).add_to(mitte_map)
#create a marker for each listing
for row in entire_homes_above500_geo.iterrows():
    row_values = row[1]
    location = [row_values['latitude'], 
                row_values['longitude']] 
    
    price = row_values['price']
    
    if(price >= 1000):
        color = 'red'
    else:
        color = 'orange'
        
    marker = folium.CircleMarker(location = location,
                                 tooltip = '£'+ str(price),
                                 radius=12, 
                                 color = None,
                                 fill_color = color,
                                fill_opacity=0.9)
    
    marker.add_to(mitte_map)
display(mitte_map)

# Joining spatial data

**Focus area:** listings that are within the mitte including those at the boarder


**Sjoin Arguments**
sjoin.() has two core arguments: how and op.

**op argument:** specifies how geopandas decides whether or not to join the attributes of one object to another
There are three different join options as follows:
   - **intersects:** The attributes will be joined if the boundary and interior of the object intersect in any way with the boundary and/or interior of the other object.
   - **within:** The attributes will be joined if the object’s boundary and interior intersect only with the interior of the other object (not its boundary or exterior).
   - **contains:** The attributes will be joined if the object’s interior contains the boundary and interior of the other object and their boundaries do not touch at all.

**how argument:** specifies the type of join that will occur and which geometry is retained in the resultant geodataframe. 

It accepts the following options:
- **left:** use the index from the first (or left_df) geodataframe that you provide to sjoin; retain only the left_df geometry column
- **right:** use index from second (or right_df); retain only the right_df geometry column
- **inner:** use intersection of index values from both geodataframes; retain only the left_df geometry column

In [ ]:
# join spatial data considering only the data with in Mitte area
mitte_listings_in_mitte_geo = gpd.sjoin(entire_homes_above500_geo, 
                               berlin_mitte, 
                               how='inner', 
                               op = 'within') 
mitte_listings_in_mitte_geo.shape

# HTML Popup annotations

Scenario: for each listing show the name of the surburb and the price

In [ ]:
import re

# Add description on markers
mitte_map = folium.Map(location = mitte_center,
                       width='80', 
                       height='80', 
                       tiles='cartodbpositron',
                       zoom_start = 12)


# Construct a folium map for listing in mitte only 
folium.GeoJson(berlin_mitte.geometry).add_to(mitte_map)

#create a marker for each listing
for row in mitte_listings_in_mitte_geo.iterrows():   
    row_values = row[1]
    price = row_values['price']
    name = re.sub('[^A-Za-z0-9 ]', '', row_values['name_left'])
    location = [row_values['latitude'], row_values['longitude']] 
    popuptext = str(name) +'<br> <strong> £' + str(price) + '</strong>'
    
    popup = folium.Popup(popuptext,max_width=100,min_width=50)
    
    if(price >= 1000):
        icon = folium.Icon(color='red', icon='home')
    else:
        icon = folium.Icon(color='lightred', icon='home')
        
    marker = folium.Marker(location = location, popup = popup,icon=icon) 
    marker.add_to(mitte_map)
display(mitte_map)


# Heat (Density) maps

To plot a heatmap of listing in each neigborhood_group, we will need to compute the density of listing in each neighorhood_group.

**Therefore we will:**
- Calculate area for each polygon
- Count listings in neigborhood_group
- Use the area and total listings to compute density of listings per square Kilometer


In [ ]:
#Checking the dataset
berlin_heatmap_gdf = berlin[['name','geometry']]
berlin_heatmap_gdf.head(2)

In [ ]:
type(berlin_heatmap_gdf)

## Calculate area for each polygon

To calucate area we need to convert coordinates from degrees to distance

In [ ]:
print(berlin_heatmap_gdf.crs)

In [ ]:
# convert to EPSG 3857 to calculate area
berlin_heatmap_gdf = berlin_heatmap_gdf.to_crs(epsg = 3857) 
print(berlin_heatmap_gdf.crs)


In [ ]:
#add area column in square kilometers
berlin_heatmap_gdf['area']=berlin_heatmap_gdf.geometry.area / (10**6)
berlin_heatmap_gdf.head(2)

In [ ]:
# convert back to EPSG 4326 the default crs for OpenStreetMap
berlin_heatmap_gdf = berlin_heatmap_gdf.to_crs(epsg = 4326) 

In [ ]:
berlin_heatmap_gdf.head()

## Grouping listings by neigborhood group
On comparison of two datasets the map structure is by 'neigborhood_group', so we will group by that.

In [ ]:
# aggregate listing by neighorhood
listings_counts = df_listings.groupby(['neighbourhood_group']).size()

# convert listings_counts to a df
berlin_listings_df = listings_counts.to_frame() 
berlin_listings_df.reset_index(level=0, inplace=True) 
berlin_listings_df.columns = ['name', 'listings_count']
berlin_listings_df.head(2)


### Merging DataFrames:
**how** : {‘left’, ‘right’, ‘outer’, ‘inner’}, default ‘inner’

**how options:**
Specifies the type of join that will occur 
- left: use only keys from left frame, similar to a SQL left outer join; preserve key order.
- right: use only keys from right frame, similar to a SQL right outer join; preserve key order.
- outer: use union of keys from both frames, similar to a SQL full outer join; sort keys lexicographically.
- inner: use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys.

**on** : label or list

Specifies the column or index level names to join on. 
- These must be found in both DataFrames. 
- If on is None and not merging on indexes then this defaults to the intersection of the columns in both DataFrames.

In [ ]:
# merge
berlin_listings_with_counts = pd.merge(berlin_heatmap_gdf, 
                                       berlin_listings_df, 
                                       how='inner', 
                                       on = 'name')
berlin_listings_with_counts.head(2)

# Calculating density of listings

In [ ]:
print(berlin_listings_df.shape, type(berlin_listings_df))

In [ ]:
berlin_listings_with_counts['listings_density'] = berlin_listings_with_counts.apply(
                                                        lambda row: ((row.listings_count/row.area)), axis = 1)
berlin_listings_with_counts.head(2)



In [ ]:
berlin_listings_with_counts.plot(column = 'listings_density', cmap = 'Reds', edgecolor = 'black', legend = True)
plt.title('Listings per kilometers squared') 
plt.xlabel('longitude') 
plt.ylabel('latitude');

In [ ]:

berlin_listings_with_counts_geo = gpd.GeoDataFrame(berlin_listings_with_counts, 
                                                   crs = listings_crs, 
                                                   geometry = berlin_listings_with_counts.geometry)
berlin_listings_with_counts_geo.head(1)

# Working with folium choropleth

**Arguments:**
- **geo_data** - the source data for the polygons (geojson file or a GeoDataFrame) 
- **name** - the name of the geometry column (or geojson property) for the polygons 
- **data** - the source DataFrame or Series for the normalized data
- **columns** - a list of columns: one that corresponds to the polygons and one that has the value to plot

**Additional arguments**

- **key_on** - a GeoJSON variable to bind the data to (always starts with feature) 
- **fill_color** - polygon fill color (defaults to blue)
- **fill_opacity** - range between 0 (transparent) and 1 (completely opaque) 
- **line_color** - color of polygon border lines (defaults to black)
- **line_opacity** - range between 0 (transparent) and 1 (completely opaque) 
- **legend_name** - creates a title for the legend

In [ ]:
# Center point and map for Berlin
berlin_map_center = [52.532893752113054, 13.365964300254726]
m = folium.Map(location=berlin_map_center, 
               zoom_start=10,
               width='80', 
               height='80')

In [ ]:
# Define a choropleth layer for the map
m.choropleth(
    geo_data=berlin_listings_with_counts_geo,
    name='geometry',
    data=berlin_listings_with_counts_geo,
    columns=['name', 'listings_density'], 
    key_on='feature.properties.name', 
    fill_color='Reds',
    fill_opacity=0.8,
    line_opacity=0.4,
    legend_name='Airbnb listings per Squared Km'
)

In [ ]:
# Add layer control and display
folium.LayerControl().add_to(m) 
display(m)